## Setup

In [ ]:
# Install Pytorch & other libraries
%pip install "torch>=2.4.0" tensorboard

# Install Gemma release branch from Hugging Face
%pip install "transformers>=4.51.3"

%pip install gguf

# Install Hugging Face libraries
%pip install  --upgrade \
  "datasets==3.3.2" \
  "accelerate==1.4.0" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.45.3" \
  "trl==0.15.2" \
  "peft==0.14.0" \
  protobuf \
  sentencepiece

# COMMENT IN: if you are running on a GPU that supports BF16 data type and flash attn, such as NVIDIA L4 or NVIDIA A100
%pip install flash-attn

  Using cached flash_attn-2.7.4.post1.tar.gz (6.0 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [ ]:
import os
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
from peft import PeftModel
import torch

In [ ]:
from huggingface_hub import login

login()

## Loading the model

In [ ]:
# Hugging Face model id
model_id = "google/gemma-2-2b-it"

# Select model class based on id
model_class = AutoModelForCausalLM

# Check if GPU benefits from bfloat16
if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
    torch_dtype=torch_dtype, # What torch dtype to use, defaults to auto
    device_map="auto", # Let torch decide how to load the model
)

# BitsAndBytesConfig: Enables 4-bit quantization to reduce model size/memory usage
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['torch_dtype'],
    bnb_4bit_quant_storage=model_kwargs['torch_dtype'],
)

# Load model and tokenizer
model = model_class.from_pretrained(model_id, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it") # Load the Instruction Tokenizer to use the official Gemma template


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

## Preparing the data

In [ ]:
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [ ]:
def prep_dataset(items):
  converted_sample = [
      {"role": "user", "content": f'You are a helpful and competent SQL Query writer, please turn the following English-language request, marked by <REQUEST> tags, and turn it into a semantically equivalent, syntactically correct SQL query.\n<REQUEST>{items["sql_prompt"]}</REQUEST>'},
      {"role": "assistant", "content": items["sql"]},
      ]

  return {'messages': converted_sample}

In [ ]:
subdataset = dataset.shuffle(seed=142).select(range(5000))

In [ ]:
prepped_dataset = subdataset.map(prep_dataset).remove_columns([
    'id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'])
prepped_dataset

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 5000
})

## Fine-tuning the model

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj', 'k_proj', 'q_proj', 'o_proj',
        'gate_proj', 'up_proj', 'down_proj'],
)

In [ ]:
sft_config = SFTConfig(
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    gradient_accumulation_steps=2,
    per_device_train_batch_size=128,
    auto_find_batch_size=True,

    max_seq_length=512,
    packing=True,

    num_train_epochs=2,
    learning_rate=2e-4,
    lr_scheduler_type="constant",
    optim='adamw_torch_fused',
    max_grad_norm=0.3,
    warmup_ratio=0.03,

    logging_steps=10,
    logging_dir='./logs',
    output_dir='./gemma-3-texttosql-adapter-with_instrction_v2',
    report_to='none',
    push_to_hub=False
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    peft_config=peft_config,
    train_dataset=prepped_dataset,
)

Converting train dataset to ChatML:   0%|          | 0/5000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.718200
20,0.835900
30,0.610700
40,0.548500
50,0.513000
60,0.493100


TrainOutput(global_step=68, training_loss=0.7509933254298042, metrics={'train_runtime': 1061.9056, 'train_samples_per_second': 2.057, 'train_steps_per_second': 0.064, 'total_flos': 1.3423045581471744e+16, 'train_loss': 0.7509933254298042})

In [ ]:
trainer.save_model()

In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

## Registry

### Merging the adapter and the model

In [ ]:
# Load Model base model
model = model_class.from_pretrained(model_id, low_cpu_mem_usage=True)

# Merge LoRA and base model and save
peft_model = PeftModel.from_pretrained(model, sft_config.output_dir)
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained(
    "gemma-2_2B-texttosql-with_instrction-merged",
    safe_serialization=True, max_shard_size="2GB")

processor = AutoTokenizer.from_pretrained(sft_config.output_dir)
processor.save_pretrained(
    "gemma-2_2B-texttosql-with_instrction-merged")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('gemma-2_2B-texttosql-with_instrction-merged/tokenizer_config.json',
 'gemma-2_2B-texttosql-with_instrction-merged/special_tokens_map.json',
 'gemma-2_2B-texttosql-with_instrction-merged/tokenizer.model',
 'gemma-2_2B-texttosql-with_instrction-merged/added_tokens.json',
 'gemma-2_2B-texttosql-with_instrction-merged/tokenizer.json')

In [ ]:
print(f"Saving to hub ...")
merged_model.push_to_hub(
    "gemma-2_2B-texttosql-with_instrction-merged",
    use_temp_dir=False, private=False)
processor.push_to_hub(
    "gemma-2_2B-texttosql-with_instrction-merged",
    use_temp_dir=False, private=False)

Saving to hub ...


model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/szaboildi/gemma-2_2B-texttosql-with_instrction-merged/commit/fc92f9d35abd0b9b6a1bdc9f879158287ec5d42d', commit_message='Upload tokenizer', commit_description='', oid='fc92f9d35abd0b9b6a1bdc9f879158287ec5d42d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/szaboildi/gemma-2_2B-texttosql-with_instrction-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='szaboildi/gemma-2_2B-texttosql-with_instrction-merged'), pr_revision=None, pr_num=None)

### GGUF

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 51370, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 51370 (delta 177), reused 63 (delta 62), pack-reused 51095 (from 2)
Receiving objects: 100% (51370/51370), 117.07 MiB | 38.46 MiB/s, done.
Resolving deltas: 100% (37090/37090), done.


In [ ]:
!pip install -r llama.cpp/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.6 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py gemma-2_2B-texttosql-with_instrction-merged --outfile gemma-2_2B-texttosql-with_instrction-merged_gguf_q8.gguf --outtype q8_0

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
INFO:hf-to-gguf:Loading model: gemma-2_2B-texttosql-with_instrction-merged
INFO:hf-to-gguf:Model architecture: Gemma2ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00006.safetensors'
INFO:hf-to-gguf:token_embd.weight,                 torch.float32 --> Q8_0, shape = {2304, 256000}
INFO:hf-to-gguf:gguf: loading model part 'model-00002-of-00006.safetensors'
INFO:hf-to-gguf:blk.0.attn_norm.weight,            torch.float32 --> F32, shape = {2304}
INFO:hf-to-gguf:blk.0.ffn_down.weight,             torch.float32 --> Q8_0, shape = {9216, 2304

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py gemma-2_2B-texttosql-with_instrction-merged --outfile gemma-2_2B-texttosql-with_instrction-merged_gguf_f16.gguf --outtype f16

INFO:hf-to-gguf:Loading model: gemma-2_2B-texttosql-with_instrction-merged
INFO:hf-to-gguf:Model architecture: Gemma2ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00006.safetensors'
INFO:hf-to-gguf:token_embd.weight,                 torch.float32 --> F16, shape = {2304, 256000}
INFO:hf-to-gguf:gguf: loading model part 'model-00002-of-00006.safetensors'
INFO:hf-to-gguf:blk.0.attn_norm.weight,            torch.float32 --> F32, shape = {2304}
INFO:hf-to-gguf:blk.0.ffn_down.weight,             torch.float32 --> F16, shape = {9216, 2304}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,             torch.float32 --> F16, shape = {2304, 9216}
INFO:hf-to-gguf:blk.0.ffn_up.weight,               torch.float32 --> F16, shape = {2304, 9216}
INFO:hf-to-gguf:blk.0.post_attention_norm.weight,  torch.floa

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "szaboildi/gemma-2_2B-texttosql-with_instrction-merged_gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj="gemma-2_2B-texttosql-with_instrction-merged_gguf_q8.gguf",
    path_in_repo="gemma-2_2B-texttosql-with_instrction-merged_gguf_q8.gguf",
    repo_id=model_id,
)

api.upload_file(
    path_or_fileobj="gemma-2_2B-texttosql-with_instrction-merged_gguf_f16.gguf",
    path_in_repo="gemma-2_2B-texttosql-with_instrction-merged_gguf_f16.gguf",
    repo_id=model_id,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


gemma-2_2B-texttosql-with_instrction-merged_gguf_q8.gguf:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

gemma-2_2B-texttosql-with_instrction-merged_gguf_f16.gguf:   0%|          | 0.00/5.24G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/szaboildi/gemma-2_2B-texttosql-with_instrction-merged_gguf/commit/795cf666c7036809398bf20e3a75ceef1024d7e9', commit_message='Upload gemma-2_2B-texttosql-with_instrction-merged_gguf_f16.gguf with huggingface_hub', commit_description='', oid='795cf666c7036809398bf20e3a75ceef1024d7e9', pr_url=None, pr_revision=None, pr_num=None)

## Testing

In [ ]:
%pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 33.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4123125 sha256=de5048deac90ed973ffea0c489e8fd4bc615906abb1a3f72a99fdcff7a12b443
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [ ]:
sentence = "How many needles are there on average in the three biggest haystacks?"
sentence2 = "What colors were apples from the last five months?"

In [ ]:
import re

def format_prompt_llama(sentence):
    prompt = f"<start_of_turn>user\nYou are a helpful and competent SQL Query writer, please turn the following English-language request, marked by <REQUEST> tags, and turn it into a semantically equivalent, syntactically correct SQL query.\n<REQUEST>{sentence}</REQUEST><end_of_turn>\n<start_of_turn>model"
    return prompt

In [ ]:
def chat(model_llama, query):
    prompt = format_prompt_llama(query)

    output = model_llama(
        prompt, max_tokens=128, echo=True)
    print("###########################################################")
    reply = re.search(
        r'<start_of_turn>model\n(.*?;)',
        output["choices"][0]["text"])

    if reply is not None:
        return reply.group(1)

    return output["choices"][0]["text"]


In [ ]:
from llama_cpp import Llama

filename = "gemma-2_2B-texttosql-with_instrction-merged_gguf_q8.gguf"
model = Llama(model_path=filename)

llama_model_loader: loaded meta data with 33 key-value pairs and 288 tensors from gemma-2_2B-texttosql-with_instrction-merged_gguf_q8.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gemma 2_2B Texttosql With_Instrction ...
llama_model_loader: - kv   3:                           general.finetune str              = texttosql-with_instrction-merged
llama_model_loader: - kv   4:                           general.basename str              = gemma
llama_model_loader: - kv   5:                         general.size_label str              = 2.2B
llama_model_loader: - kv   6:                      gemma2.context_length u32          

In [ ]:
print(chat(model, sentence))

llama_perf_context_print:        load time =    1607.97 ms
llama_perf_context_print: prompt eval time =    1607.68 ms /    70 tokens (   22.97 ms per token,    43.54 tokens per second)
llama_perf_context_print:        eval time =    1921.45 ms /    26 runs   (   73.90 ms per token,    13.53 tokens per second)
llama_perf_context_print:       total time =    3596.36 ms /    96 tokens


###########################################################
SELECT AVG(height) FROM haystacks WHERE height IN (SELECT height FROM haystacks ORDER BY height DESC LIMIT 3);


In [ ]:
print(chat(model, sentence2))

Llama.generate: 50 prefix-match hit, remaining 16 prompt tokens to eval
llama_perf_context_print:        load time =    1607.97 ms
llama_perf_context_print: prompt eval time =     226.87 ms /    16 tokens (   14.18 ms per token,    70.52 tokens per second)
llama_perf_context_print:        eval time =    3013.98 ms /    39 runs   (   77.28 ms per token,    12.94 tokens per second)
llama_perf_context_print:       total time =    3340.10 ms /    55 tokens


###########################################################
SELECT color FROM apples WHERE harvest_date >= NOW() - INTERVAL '5 months' AND id IN (SELECT id FROM apples WHERE harvest_date >= NOW() - INTERVAL '5 months');
